**Universidade Federal Rural de Pernambuco**  
Departamento de Computação  
Disciplina: Inteligência Artificial  
Docente: André Câmara  
Discentes: Edilson Alves e Marcelino Chagas  

### <center>Projeto 2ª VA - Aprendizagem de Máquina</center>
### <center>Sistema de Recomendação de Filmes</center>

### Preparação dos Dados

#### 01. Carregamento

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ratings = pd.read_csv('ratings.csv', sep='\t', encoding='latin-1', usecols=['user_id', 'movie_id', 'rating'])
users = pd.read_csv('users.csv', sep='\t', encoding='latin-1', usecols=['user_id', 'gender', 'zipcode', 'age_desc', 'occ_desc'])
movies = pd.read_csv('movies.csv', sep='\t', encoding='latin-1', usecols=['movie_id', 'title', 'genres'])

#### 02. Visualização

In [0]:
print(ratings.head())

   user_id  movie_id  rating
0        1      1193       5
1        1       661       3
2        1       914       3
3        1      3408       4
4        1      2355       5


In [0]:
print(users.head())

   user_id gender zipcode  age_desc              occ_desc
0        1      F   48067  Under 18          K-12 student
1        2      M   70072       56+         self-employed
2        3      M   55117     25-34             scientist
3        4      M   02460     45-49  executive/managerial
4        5      M   55455     25-34                writer


In [0]:
print(movies.head())

   movie_id                               title                        genres
0         1                    Toy Story (1995)   Animation|Children's|Comedy
1         2                      Jumanji (1995)  Adventure|Children's|Fantasy
2         3             Grumpier Old Men (1995)                Comedy|Romance
3         4            Waiting to Exhale (1995)                  Comedy|Drama
4         5  Father of the Bride Part II (1995)                        Comedy


### Modelo de Recomendação Baseado em Conteúdo

#### 01. Implementação

In [0]:
movies['genres'] = movies['genres'].str.split('|')
movies['genres'] = movies['genres'].fillna("").astype('str')

In [0]:
# Transforma o texto bruto em uma matrix de recursos do TD-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres'])
tfidf_matrix.shape

(3883, 127)

In [0]:
# Calcula a similaridade dos valores da matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim[:4, :4]

array([[1.        , 0.14193614, 0.09010857, 0.1056164 ],
       [0.14193614, 1.        , 0.        , 0.        ],
       [0.09010857, 0.        , 1.        , 0.1719888 ],
       [0.1056164 , 0.        , 0.1719888 , 1.        ]])

In [0]:
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])

def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    
    return titles.iloc[movie_indices]

In [0]:
genre_recommendations('Toy Story (1995)').head(20)

1050               Aladdin and the King of Thieves (1996)
2072                             American Tail, An (1986)
2073           American Tail: Fievel Goes West, An (1991)
2285                            Rugrats Movie, The (1998)
2286                                 Bug's Life, A (1998)
3045                                   Toy Story 2 (1999)
3542                                Saludos Amigos (1943)
3682                                   Chicken Run (2000)
3685       Adventures of Rocky and Bullwinkle, The (2000)
236                                 Goofy Movie, A (1995)
12                                           Balto (1995)
241                               Gumby: The Movie (1995)
310                             Swan Princess, The (1994)
592                                      Pinocchio (1940)
612                                Aristocats, The (1970)
700                               Oliver & Company (1988)
876     Land Before Time III: The Time of the Great Gi...
1010          

### Collaborative Filtering Recommendation Model

#### 01. Implementação

In [0]:
ratings['user_id'] = ratings['user_id'].fillna(0)
ratings['movie_id'] = ratings['movie_id'].fillna(0)
ratings['rating'] = ratings['rating'].fillna(ratings['rating'].mean())

In [0]:
# Amostra Aleatória dos dados
from sklearn.model_selection import train_test_split

# Verificando as informações da Amostra
small_data = ratings.sample(frac=0.02)
train_data, test_data = train_test_split(small_data, test_size=0.2)

In [0]:
print(small_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11132 entries, 447267 to 23104
Data columns (total 3 columns):
user_id     11132 non-null int64
movie_id    11132 non-null int64
rating      11132 non-null int64
dtypes: int64(3)
memory usage: 347.9 KB
None


In [0]:
print(train_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8905 entries, 43769 to 252909
Data columns (total 3 columns):
user_id     8905 non-null int64
movie_id    8905 non-null int64
rating      8905 non-null int64
dtypes: int64(3)
memory usage: 278.3 KB
None


In [0]:
print(test_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2227 entries, 373719 to 467788
Data columns (total 3 columns):
user_id     2227 non-null int64
movie_id    2227 non-null int64
rating      2227 non-null int64
dtypes: int64(3)
memory usage: 69.6 KB
None


In [0]:
# Dividir o dados dos arquivo em dois conjuntos 
# Um para teste e outro para treinamento

# 
train_data_matrix = train_data.as_matrix(columns = ['user_id', 'movie_id', 'rating'])
test_data_matrix = test_data.as_matrix(columns = ['user_id', 'movie_id', 'rating'])

print(train_data_matrix.shape)
print(test_data_matrix.shape)

(8905, 3)
(2227, 3)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [0]:
# Coeficiente de Pearson
# Para obter Matrizes de distancia como entrada
from sklearn.metrics.pairwise import pairwise_distances

# Correlação de Usuario
user_correlation = 1 - pairwise_distances(train_data, metric='correlation')
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation[:4, :4])

[[1.         0.4651499  0.99760575 0.77747625]
 [0.4651499  1.         0.52525674 0.91837625]
 [0.99760575 0.52525674 1.         0.81910884]
 [0.77747625 0.91837625 0.81910884 1.        ]]


In [0]:
# Correlação de Item
item_correlation = 1 - pairwise_distances(train_data_matrix.T, metric='correlation')
item_correlation[np.isnan(item_correlation)] = 0
print(item_correlation[:4, :4])

[[ 1.00000000e+00  5.74491964e-04  8.43611291e-03]
 [ 5.74491964e-04  1.00000000e+00 -7.14301791e-02]
 [ 8.43611291e-03 -7.14301791e-02  1.00000000e+00]]


Para o caso CF usuário-usuário, será observado a similaridade entre dois usuários (A e B, por exemplo) como pesos que são multiplicados pelas classificações de um usuário semelhante B (corrigido para a classificação média desse usuário). Também será necessário normalizá-lo para que as classificações fiquem entre 1 e 5 e, como etapa final, some as classificações médias do usuário que se está tentando prever. A ideia aqui é que alguns usuários tendem a sempre dar classificações altas ou baixas a todos os filmes. A diferença relativa nas classificações que esses usuários fornecem é mais importante que os valores absolutos.

In [0]:
# Funcao para prever classificacoes
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        # Use np.newaxis para que mean_user_rating tenha o mesmo formato das classificacoes
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    
    return pred

### Avaliação

In [0]:
# Funcao para calcular o RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(pred, actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    
    return sqrt(mean_squared_error(pred, actual))

In [0]:
user_prediction = predict(train_data_matrix, user_correlation, type='user')
item_prediction = predict(train_data_matrix, item_correlation, type='item')

In [0]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, train_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, train_data_matrix)))

User-based CF RMSE: 485.54641905164584
Item-based CF RMSE: 113.82270163462687


In [0]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 1028.4185661656109
Item-based CF RMSE: 1189.5282868195768
